In [35]:
import pandas as pd
df=pd.read_csv("sms.csv")
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [36]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [37]:
df_ham=df[df['label']=='ham']    # data frame contain only ham

In [39]:
df_spam=df[df['label']=='spam']   # data frame contains only spam

In [40]:
df_ham_downsample=df_ham.sample(df_spam.shape[0])   # we make 747 sample of ham..... df_spam.shape[0] gives 747

In [42]:
df_balanced= pd.concat([df_spam,df_ham_downsample])  # 747+747= 1494
df_balanced.shape

(1494, 2)

In [44]:
df_balanced['label'].value_counts()   # undersampling is done

spam    747
ham     747
Name: label, dtype: int64

In [45]:
df_balanced['label'] = df.label.map({'ham':0, 'spam':1})

In [48]:
df_balanced.sample(10)

,label,message
5453,0,Except theres a chick with huge boobs.
4104,1,U have a secret admirer who is looking 2 make ...
305,1,SMS. ac Blind Date 4U!: Rodds1 is 21/m from Ab...
1674,1,Monthly password for wap. mobsi.com is 391784....
1146,1,"Thank you, winner notified by sms. Good Luck! ..."
2688,0,Okie
5405,0,So how many days since then?
3385,1,FREE MSG:We billed your mobile number by mista...
2300,1,Congrats! 1 year special cinema pass for 2 is ...
1521,1,"URGENT! Your Mobile No was awarded a £2,000 Bo..."


In [63]:
x=df_balanced.message
y=df_balanced.label

In [64]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y= train_test_split(x,y,test_size=0.3,random_state=100,shuffle =True,stratify=df_balanced['label'])

In [65]:
train_y.value_counts()

1    523
0    522
Name: label, dtype: int64

In [66]:
pip install tensorflow-text

In [67]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [68]:
bert_preprocess=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [69]:
bert_encoder=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [70]:
def get_sentence_encoding(sentences):
  preprocessed_text= bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']
get_sentence_encoding(["I love yawar","my love for football is much important",'india is my country'])

<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[-0.90996456, -0.42290202, -0.62535894, ..., -0.33263472,
        -0.7204885 ,  0.9165213 ],
       [-0.78604144, -0.41154942, -0.65878624, ..., -0.27214226,
        -0.6124459 ,  0.8411293 ],
       [-0.86772424, -0.3965563 , -0.37748134, ..., -0.11730512,
        -0.6479711 ,  0.8981817 ]], dtype=float32)>

In [71]:
#Bert Layers
text_input=tf.keras.layers.Input(shape=(), dtype=tf.string,name='text')
preprocessed_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocessed_text)
# neural Netwok layers
dropout_output=tf.keras.layers.Dropout(0.1,name="Dropout")(outputs['pooled_output'])
dense_output=tf.keras.layers.Dense(1, activation='sigmoid', name='output')(dropout_output)
# construct a final model
model=tf.keras.Model(inputs=[text_input],outputs=[dense_output] )
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_4 (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                              

In [72]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

# Compile the model
model.compile(
    optimizer='adam',  # Optimizer
    loss='binary_crossentropy',  # Loss function
    metrics=METRICS  # Metrics to monitor during training
)

In [73]:
model.fit(train_x,train_y, epochs=10)

Epoch 1/10
33/33 [==============================] - 312s 9s/step - loss: 0.6397 - accuracy: 0.6603 - precision: 0.6750 - recall: 0.6195
Epoch 2/10
33/33 [==============================] - 307s 9s/step - loss: 0.5238 - accuracy: 0.7837 - precision: 0.7610 - recall: 0.8279
Epoch 3/10
33/33 [==============================] - 307s 9s/step - loss: 0.4456 - accuracy: 0.8565 - precision: 0.8499 - recall: 0.8662
Epoch 4/10
33/33 [==============================] - 307s 9s/step - loss: 0.3893 - accuracy: 0.8909 - precision: 0.8808 - recall: 0.9044
Epoch 5/10
33/33 [==============================] - 307s 9s/step - loss: 0.3592 - accuracy: 0.8871 - precision: 0.8610 - recall: 0.9235
Epoch 6/10
33/33 [==============================] - 307s 9s/step - loss: 0.3369 - accuracy: 0.8919 - precision: 0.8810 - recall: 0.9063
Epoch 7/10
33/33 [==============================] - 307s 9s/step - loss: 0.3124 - accuracy: 0.8976 - precision: 0.8866 - recall: 0.9120
Epoch 8/10
33/33 [==============================

In [74]:
model.evaluate(test_x,test_y)

15/15 [==============================] - 133s 9s/step - loss: 0.2806 - accuracy: 0.9154 - precision: 0.9043 - recall: 0.9286


[0.2806072235107422, 0.9153674840927124, 0.904347836971283, 0.9285714030265808]

In [76]:
y_predict=model.predict(test_x)
y_predict

15/15 [==============================] - 132s 9s/step


array([[0.50536644],
       [0.08318744],
       [0.18208736],
       [0.0242416 ],
       [0.62398314],
       [0.22278075],
       [0.3510194 ],
       [0.7821381 ],
       [0.10124369],
       [0.27913958],
       [0.02200058],
       [0.09690693],
       [0.64788824],
       [0.06406814],
       [0.04441855],
       [0.2970855 ],
       [0.44535646],
       [0.21447326],
       [0.7808685 ],
       [0.04314357],
       [0.8761789 ],
       [0.5184675 ],
       [0.9548628 ],
       [0.1364026 ],
       [0.3862578 ],
       [0.5249695 ],
       [0.59932935],
       [0.27330834],
       [0.05864874],
       [0.8378402 ],
       [0.02397267],
       [0.02673458],
       [0.26248983],
       [0.60807306],
       [0.06217833],
       [0.54691565],
       [0.18186797],
       [0.9422161 ],
       [0.86903536],
       [0.23029742],
       [0.06698689],
       [0.11151575],
       [0.12959805],
       [0.0227939 ],
       [0.8214202 ],
       [0.13804778],
       [0.5062823 ],
       [0.814

In [77]:
y_predict.flatten()

array([0.50536644, 0.08318744, 0.18208736, 0.0242416 , 0.62398314,
       0.22278075, 0.3510194 , 0.7821381 , 0.10124369, 0.27913958,
       0.02200058, 0.09690693, 0.64788824, 0.06406814, 0.04441855,
       0.2970855 , 0.44535646, 0.21447326, 0.7808685 , 0.04314357,
       0.8761789 , 0.5184675 , 0.9548628 , 0.1364026 , 0.3862578 ,
       0.5249695 , 0.59932935, 0.27330834, 0.05864874, 0.8378402 ,
       0.02397267, 0.02673458, 0.26248983, 0.60807306, 0.06217833,
       0.54691565, 0.18186797, 0.9422161 , 0.86903536, 0.23029742,
       0.06698689, 0.11151575, 0.12959805, 0.0227939 , 0.8214202 ,
       0.13804778, 0.5062823 , 0.8146161 , 0.48456177, 0.11131658,
       0.4752807 , 0.01058677, 0.7042126 , 0.9464497 , 0.81349397,
       0.03147515, 0.90857583, 0.08779031, 0.90057224, 0.43931264,
       0.4905864 , 0.11769951, 0.9665374 , 0.7479756 , 0.79718184,
       0.05046359, 0.08694506, 0.7922067 , 0.1608488 , 0.93209165,
       0.8514925 , 0.16670811, 0.73877555, 0.8985451 , 0.05808

In [78]:
import numpy as np
y_predict=np.where(y_predict>0.5,1,0)

In [80]:
from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(test_y,y_predict)
cm

array([[203,  22],
       [ 16, 208]])

In [81]:
print(classification_report(test_y,y_predict))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91       225
           1       0.90      0.93      0.92       224

    accuracy                           0.92       449
   macro avg       0.92      0.92      0.92       449
weighted avg       0.92      0.92      0.92       449

